# Descargar automáticamente de CAMMESA
A partir de la ejecución del siguiente código: 
1. Se descargan los archivos de datos cada 5 minutos de Despacho de Renovables y cubrimiento de la demanda de CAMMESA.
2. Se estandarizan los mismos.
3. Se los exporta como archivo CSV.

Fuente: https://despachorenovables.cammesa.com/renovables/

In [1]:
#Cargar librerías
import requests
import pandas as pd
import datetime as dt
from datetime import date, datetime, timedelta
from os import mkdir, rmdir, remove, path

In [2]:
# Definir
sdate = date(2020, 6, 1)   # fecha de inicio (año, mes, día)
edate = date(2020, 6, 7)   # fecha de fin (año, mes, día)

file = 'renovables.csv' #nombre del archivo de destino

In [ ]:
# Descarga de datos
dir = "renovables" #carpeta de destino de los archivos
mkdir(str(dir))

delta = edate - sdate

for i in range(delta.days + 1):
    date = (sdate + timedelta(days=i))
    date1 = date.strftime('%d-%m-%Y')
    date2 = (date + timedelta(days=1))
    date2 = date2.strftime('%d-%m-%Y')
    url = 'https://cdsrenovables.cammesa.com/exhisto/RenovablesService/GetChartTotalTRDataSource?desde=' + str(date1) + '&hasta=' + str(date2)
    r = requests.get(url, allow_redirects=True)
    if 'NOT_FOUND' in r.content.decode():
        print(str(date) + ' not Found.')  
    else:
        with open('./' + dir + '/' + str(date) + '.json', 'wb') as f: f.write(r.content)

In [ ]:
#Combinación de archivos
out = pd.DataFrame(columns = ['momento', 'eolica', 'fotovoltaica', 'biocombustible', 'hidraulica', 'cubrimientoDem'])

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i))
    if path.isfile('./' + dir + '/'  + date + '.json'):
        df = pd.read_json('./' + dir + '/'  + date + '.json')
        out = out.append(df, ignore_index=True)
    else:
        print(date + " does not exist")

display(out.head())
display(out.tail())

In [ ]:
# estandarizar datos
df = out

df['momento'] = pd.to_datetime(df.momento)
df['momento'] = df['momento'].dt.strftime('%d/%m/%Y %H:%M')
df.sort_values(by = 'momento', ascending=True, inplace=True, kind='quicksort', na_position='last')
df = df.reset_index(drop=True)

display(out.head())
display(out.tail())

In [ ]:
#Borra archivos anteriores y exporta la tabla
df.to_csv(file, index = False) 

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i))
    if path.isfile('./' + dir + '/'  + date + '.json'):
        remove('./' + dir + '/' + date + '.json')
    else:
        print(date + " does not exist")
rmdir(str(dir))